# Extract and Transform Data

#### In this notebook we will be extracting movie related information from the OMDB API and save a csv file.

1. Data source:
2. API Documentation:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json
import random
from pprint import pprint
from config import LOCAL_API_KEY
import time

In [2]:
# Visualize more columns
pd.options.display.max_columns = None

## Initial clean of the downloaded CSV

In [3]:
# Import downloaded CSV which contains a list 6820 movies
file_path = '../resources/raw_data/movies.csv'
kaggle_df = pd.read_csv(file_path, sep=',', engine='python')
kaggle_df.head()

,budget,company,country,director,genre,gross,name,rating,released,runtime,score,star,votes,writer,year
0,8000000.0,Columbia Pictures Corporation,USA,Rob Reiner,Adventure,52287414.0,Stand by Me,R,1986-08-22,89,8.1,Wil Wheaton,299174,Stephen King,1986
1,6000000.0,Paramount Pictures,USA,John Hughes,Comedy,70136369.0,Ferris Bueller's Day Off,PG-13,1986-06-11,103,7.8,Matthew Broderick,264740,John Hughes,1986
2,15000000.0,Paramount Pictures,USA,Tony Scott,Action,179800601.0,Top Gun,PG,1986-05-16,110,6.9,Tom Cruise,236909,Jim Cash,1986
3,18500000.0,Twentieth Century Fox Film Corporation,USA,James Cameron,Action,85160248.0,Aliens,R,1986-07-18,137,8.4,Sigourney Weaver,540152,James Cameron,1986
4,9000000.0,Walt Disney Pictures,USA,Randal Kleiser,Adventure,18564613.0,Flight of the Navigator,PG,1986-08-01,90,6.9,Joey Cramer,36636,Mark H. Baker,1986


In [4]:
# Describe
kaggle_df.describe()

,budget,gross,runtime,score,votes,year
count,6.820000e+03,6.820000e+03,6820.00000,6820.000000,6.820000e+03,6820.000000
mean,2.458113e+07,3.349783e+07,106.55132,6.374897,7.121952e+04,2001.000293
std,3.702254e+07,5.819760e+07,18.02818,1.003142,1.305176e+05,8.944501
min,0.000000e+00,7.000000e+01,50.00000,1.500000,2.700000e+01,1986.000000
25%,0.000000e+00,1.515839e+06,95.00000,5.800000,7.665250e+03,1993.000000
50%,1.100000e+07,1.213568e+07,102.00000,6.400000,2.589250e+04,2001.000000
75%,3.200000e+07,4.006534e+07,115.00000,7.100000,7.581225e+04,2009.000000
max,3.000000e+08,9.366622e+08,366.00000,9.300000,1.861666e+06,2016.000000


In [5]:
# Number of rows
len(kaggle_df)

6820

In [6]:
# Rename a few columns to make distinctions between the Kaggle CSV and dimensions
# scraped from the OMDB API
renamed_df = kaggle_df.rename(columns={'country': 'country_kaggle',
                                       'genre': 'genre_kaggle',
                                       'writer': 'writer_kaggle',
                                       'star': 'star_kaggle',
                                       'company': 'production',
                                       'score': 'score_imdb',
                                       'votes': 'votes_imdb'})
renamed_df.head(2)

,budget,production,country_kaggle,director,genre_kaggle,gross,name,rating,released,runtime,score_imdb,star_kaggle,votes_imdb,writer_kaggle,year
0,8000000.0,Columbia Pictures Corporation,USA,Rob Reiner,Adventure,52287414.0,Stand by Me,R,1986-08-22,89,8.1,Wil Wheaton,299174,Stephen King,1986
1,6000000.0,Paramount Pictures,USA,John Hughes,Comedy,70136369.0,Ferris Bueller's Day Off,PG-13,1986-06-11,103,7.8,Matthew Broderick,264740,John Hughes,1986


In [7]:
scraped_df = renamed_df.copy()

# Add empty columns for dimensions we are going to scrape from OMDB API
scraped_df['genres_omdb'] = ''
scraped_df['writers_omdb'] = ''
scraped_df['actors_omdb'] = ''
scraped_df['plot'] = ''
scraped_df['language_omdb'] = ''
scraped_df['country_omdb'] = ''
scraped_df['awards'] = ''
scraped_df['poster'] = ''
scraped_df['score_metacritic'] = ''
scraped_df['type'] = ''

scraped_df.head(2)

,budget,production,country_kaggle,director,genre_kaggle,gross,name,rating,released,runtime,score_imdb,star_kaggle,votes_imdb,writer_kaggle,year,genres_omdb,writers_omdb,actors_omdb,plot,language_omdb,country_omdb,awards,poster,score_metacritic,type
0,8000000.0,Columbia Pictures Corporation,USA,Rob Reiner,Adventure,52287414.0,Stand by Me,R,1986-08-22,89,8.1,Wil Wheaton,299174,Stephen King,1986,,,,,,,,,,
1,6000000.0,Paramount Pictures,USA,John Hughes,Comedy,70136369.0,Ferris Bueller's Day Off,PG-13,1986-06-11,103,7.8,Matthew Broderick,264740,John Hughes,1986,,,,,,,,,,


## Perform API Calls to OMDB API

And save extracted information in the dataframe

In [8]:
# Set up API key for requests
api_key = LOCAL_API_KEY

In [ ]:
# Use the lat/lng we recovered to identify airports

print('Beginning Data Retrieval...')
print('-----------------------------')
    
for index, row in scraped_df.iloc[2000:3000, :].iterrows():

    # Use a new movie name for each new api call
    movie_name = scraped_df.loc[index, 'name']

    # Use the search term: 'International Airport' and our lat/lng
    query_url = f'http://www.omdbapi.com/?apikey={api_key}&t={movie_name}&plot=full'

    # make request to url and conver to json
    api_data = requests.get(query_url).json()
    
    # Build some mechanisms to better 
    if index % 100 == 0:
            print(f'Processing Movie Index #{index}')
            
    if index % 250 == 0:
        time.sleep(10)
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        scraped_df.loc[index, 'genres_omdb'] = api_data['Genre']
        scraped_df.loc[index, 'writers_omdb'] = api_data['Writer']
        scraped_df.loc[index, 'actors_omdb'] = api_data['Actors']
        scraped_df.loc[index, 'plot'] = api_data['Plot']
        scraped_df.loc[index, 'language_omdb'] = api_data['Language']
        scraped_df.loc[index, 'country_omdb'] = api_data['Country']
        scraped_df.loc[index, 'awards'] = api_data['Awards']
        scraped_df.loc[index, 'poster'] = api_data['Poster']
        scraped_df.loc[index, 'score_metacritic'] = api_data['Metascore']
        scraped_df.loc[index, 'type'] = api_data['Type']
        
    except (KeyError, IndexError):
        print('Missing field/result... skipping.')

print('-----------------------------')
print('Data Retrieval Complete.') 
print('-----------------------------')

Beginning Data Retrieval...
-----------------------------
Processing Movie Index #2000
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Processing Movie Index #2100
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Processing Movie Index #2200
Missing field/result... skipping.
Missing field/result... skipping.
Processing Movie Index #2300
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Processing Movie Index #2400
Missing field/result... skipping.
Missing field/result... skipping.
Processing Movie Index #2500
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Processing Movie Index #2600
Missing field/result... s

In [ ]:
scraped_df['type'].value_counts()

In [ ]:
# scraped_df.count()
# Strategy: scrape 1000 new rows each time
test_df = scraped_df.iloc[2000:3001, :]
test_df

In [ ]:
# Testing...
# ........

# Request data
movie= 'The Five Heartbeats'
# movie= 'Jason Lives: Friday the 13th Part VI'
test_url = f'http://www.omdbapi.com/?apikey={api_key}&t={movie}&plot=full'
response = requests.get(test_url)
data = response.json()

pprint(data)

In [ ]:
pprint(data)

In [ ]:
# Export new dataset to CSV
final_df = scraped_df.copy()
final_df.to_csv('../resources/cleaned_data/movies_complete.csv', index=False)